# What is a Deep Gaussian Process?

In [ ]:
% Comment/MATLAB set up code
    importTool('dimred')
    dimredToolboxes
    randn('seed', 1e6)
    rand('seed', 1e6)
    if ~isoctave
      colordef white
    end
    patchColor = [0.8 0.8 0.8];
    colorFigs = true;
    % Text width in cm.
    textWidth = 13


In [ ]:
    # nonLinearDifficultyPlot1
    number = 200;
    numBasisFunc = 3;
    rbfWidth = 2;
    mu = linspace(-4, 4, numBasisFunc);
    x = linspace(-6, 6, number)';
    numSamples = 10;
    alpha = 1;
    W = randn(numSamples, numBasisFunc)*sqrt(alpha);
    clf
    numberAcross = 30;
    x1 = linspace(-1, 1, numberAcross);
    x2 = x1;
    mu1 = mu;
    mu2 = mu;
    [MU1, MU2] = meshgrid(mu1, mu2);
    [X1, X2] = meshgrid(x1, x2);
    X = [X1(:) X2(:)];
    MU = [MU1(:) MU2(:)];
    numBasisFunc = size(MU, 1);
    number = size(X, 1);
    Phi = exp(-dist2(X, MU)/(2*rbfWidth*rbfWidth));
    numSamples = 3;
    W = randn(numSamples, numBasisFunc)*sqrt(alpha);
    F = Phi*W';
    subplot(1, 3, 1)
    hold on
    startVal = 1;
    for i = 1:numberAcross
      endVal = numberAcross*i;
      a = plot(X(startVal:endVal, 1), X(startVal:endVal, 2), 'r-');
      startVal = endVal + 1;
    end
    % Reshape X to plot lines in opposite directions
    X1 = X1';
    X2 = X2';
    X = [X1(:) X2(:)];
    startVal = 1;
    for i = 1:numberAcross
      endVal = numberAcross*i;
      a = plot(X(startVal:endVal, 1), X(startVal:endVal, 2), 'r-');
      startVal = endVal + 1;
    end
    hold off
    axis equal
    axis off
    xlabel('$\latentScalar_1$', 'horizontalalignment', 'center')
    ylabel('$\latentScalar_2$', 'horizontalalignment', 'center')

    
    subplot(1, 3, 3)
    hold on
    startVal = 1;
    for i = 1:numberAcross
      endVal = numberAcross*i;
      a = plot3(F(startVal:endVal, 1), F(startVal:endVal, 2), F(startVal:endVal, 3), 'r-');
      startVal = endVal + 1;
    end
    % Reshape F to plot lines in opposite directions
    F1 = reshape(F(:, 1), size(X1, 1), size(X1, 2))';
    F2 = reshape(F(:, 2), size(X1, 1), size(X1, 2))';
    F3 = reshape(F(:, 3), size(X1, 1), size(X1, 2))';
    F = [F1(:) F2(:) F3(:)];
    startVal = 1;
    for i = 1:numberAcross
      endVal = numberAcross*i;
      a = plot3(F(startVal:endVal, 1), F(startVal:endVal, 2), F(startVal:endVal, 3), 'r-');
      startVal = endVal + 1;
    end
    hold off
    axis equal
    axis off
    % Treble axis size to increase plot size
    pos = get(gca, 'position');
    npos = pos;
    npos(3:4) = pos(3:4)*3;
    npos(1) = pos(1) - 0.5*(npos(3)-pos(3));
    npos(2) = pos(2) - 0.5*(npos(4)-pos(4));
    set(gca, 'position', npos);
    % pause

    % Axis for writing text on plot
    axes('position', [0 0 1 1])
    set(gca, 'xlim', [0 1])
    set(gca, 'ylim', [0 1])
    axis off
    text(0.5, 0.55, '\large$\dataScalar_j = \mappingFunction_j(\latentVector)$', 'horizontalalignment', 'center')
    text(0.5, 0.45, '\LARGE$\longrightarrow$', 'horizontalalignment', 'center')


    options = printLatexOptions;
    options.maintainAspect = false;
    options.height = 0.3*textWidth;
    printLatexPlot('nonlinearMapping3DPlot_slides', '../../../dimred/tex/diagrams', 0.95*textWidth, options);



In [ ]:
 # nonLinearDifficultyPlot2
    clf
    numberAcross = 101;
    x = linspace(-6, 6, numberAcross)';
    numBasisFunc = size(mu, 2);
    number = size(x, 1);
    Phi = exp(-dist2(x, mu')/(2*rbfWidth*rbfWidth));
    F = Phi*W(1:2, 1:numBasisFunc)';
    subplot(1, 3, 1)
    a = plot(x, ones(size(x, 1), size(x, 2)), 'r-');
    hold on
    subx = x(1:10:end);
    b = plot(subx, ones(size(subx, 1), size(subx, 2)), 'bx');
    set(gca, 'ylim', [-0.9 1.1])
    axis equal
    set(a, 'linewidth', 3);
    set(b, 'markersize', 10);
    axis off
    % Reshape X to plot lines in opposite directions
    xlabel('$\latentScalar$', 'horizontalalignment', 'center')
    
    subplot(1, 3, 3)
    a = plot(F(:, 1), F(:, 2), 'r-');
    hold on
    b = plot(F(1:10:end, 1), F(1:10:end, 2), 'bx');
    set(a, 'linewidth', 3)
    set(b, 'markersize', 10);
    xlabel('$\dataScalar_1$')
    ylabel('$\dataScalar_2$')
    hold off
    axis equal
    axis off

    % Axis for writing text on plot
    axes('position', [0 0 1 1])
    set(gca, 'xlim', [0 1])
    set(gca, 'ylim', [0 1])
    axis off
    text(0.5, 0.65, '\large$\dataScalar_1 = \mappingFunction_1(\latentScalar)$', 'horizontalalignment', 'center')
    text(0.5, 0.5, '\LARGE$\longrightarrow$', 'horizontalalignment', 'center')
    text(0.5, 0.35, '\large$\dataScalar_2 = \mappingFunction_2(\latentScalar)$', 'horizontalalignment', 'center')


    options = printLatexOptions;
    options.maintainAspect = false;
    options.height = 0.3*textWidth;
    printLatexPlot('nonlinearMapping2DPlot_slides', '../../../dimred/tex/diagrams', 0.95*textWidth, options);



In [ ]:
# nonLinearDifficultyPlot3
clf
dataStd = 0.2;
numSamples = 10000;
xsamp = randn(numSamples, 1);

# Create RBF network with much larger variation in functions.
mu = linspace(-4, 4, 100);
numBasisFunc = size(mu, 2);
rbfWidth = 0.1;
Phi = exp(-dist2(xsamp, mu')/(2*rbfWidth*rbfWidth));
W = randn(1, numBasisFunc)*sqrt(alpha);
f = Phi*W';
subplot(1, 3, 1)
hold on
p = exp(-0.5/alpha*x.^2)*1/sqrt(2*pi*alpha);
a = patch(x, p, patchColor);
axis off
set(a, 'linewidth', 2);
set(gca, 'ylim', [0 0.5]);
set(gca, 'xlim', [-6 6]);
xlabel('$p(\latentScalar)$', 'horizontalalignment', 'center')

subplot(1, 3, 3)
y = linspace (min(f)-3*dataStd, max(f)+3*dataStd, 100)';
p = mean(exp(-0.5/(dataStd*dataStd)*dist2(y, f))*1/(sqrt(2*pi)*dataStd), 2);
a = patch(y, p, patchColor);
axis off
set(a, 'linewidth', 2);
set(gca, 'ylim', [0 0.5])
set(gca, 'xlim', [-6 6]);
xlabel('$p(\dataScalar)$', 'horizontalalignment', 'center')

% Axis for writing text on plot
axes('position', [0 0 1 1])
set(gca, 'xlim', [0 1])
set(gca, 'ylim', [0 1])
axis off
text(0.5, 0.45, '\large$\dataScalar = \mappingFunction(\latentScalar) + \n oiseScalar$', 'horizontalalignment', 'center')
text(0.5, 0.35, '\LARGE$\longrightarrow$', 'horizontalalignment', 'center')

options = printLatexOptions;
options.maintainAspect = false;
options.height = 0.3*textWidth;
printLatexPlot('gaussianThroughNonlinear2_slides', '../../../dimred/tex/diagrams', 0.95*textWidth, options);
printLatexText(['\global\long\def\r bfWidthVal {' numsf2str (rbfWidth, 0) '}\global\long\def\dataStdVal{' strrep (numsf2str(dataStd, 1), ' ', '\ ') '}'], 'gaussianThroughNonlinear2Values_slides.tex', '../../../dimred/tex/diagrams');